In [1]:
import ipywidgets as widgets

In [2]:
widgets.IntProgress(
    value=7,
    min=0,
    max=10,
    step=1,
    description='Loading:',
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    orientation='horizontal'
)

IntProgress(value=7, description='Loading:', max=10)

In [3]:
widgets.FloatProgress(
    value=7.5,
    min=0,
    max=10.0,
    step=0.1,
    description='Loading:',
    bar_style='info',
    orientation='horizontal'
)

FloatProgress(value=7.5, bar_style='info', description='Loading:', max=10.0)

In [4]:
import time
from astropy.utils.console import ProgressBar

In [5]:
for i in ProgressBar(range(10)):
    time.sleep(0.1)

In [6]:
for i in ProgressBar(range(10), ipython_widget=True):
    time.sleep(0.1)

FloatProgress(value=0.0)

In [7]:
from tqdm import tqdm
from tqdm import tnrange, tqdm_notebook

In [8]:
for i in tqdm(range(10)):
    time.sleep(0.1)

100%|██████████| 10/10 [00:01<00:00,  9.77it/s]


In [9]:
for i in tqdm_notebook(range(10)):
    time.sleep(0.1)

HBox(children=(IntProgress(value=0, max=10), HTML(value='')))